# Load Data From S3 Buckets

![S3 Logo](https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/s3-logo.png  "doc-image")

## Overview

If you use AWS S3 to store your data, connecting to Saturn Cloud takes just a couple of steps. 

In this example we use <a href="https://github.com/dask/s3fs/" target='_blank' rel='noopener'>s3fs</a> to connect to data, but you can also use libraries like <a href="https://aws.amazon.com/sdk-for-python/" target='_blank' rel='noopener'>boto3</a> if you prefer.

Before starting this, you should create a Jupyter server resource. See our [quickstart](https://saturncloud.io/docs/start_in_ten/) if you don't know how to do this yet.

## Process

To connect to public S3 buckets, you can simply connect using anonymous connections in Jupyter, the way you might with your local laptop. In this case, you can skip to the [Connect to Data Via `s3fs` section](#connect-via-s3fs).

If your S3 data storage is not public and requires AWS credentials, please read on! 

### Create AWS Credentials
Credentials for S3 access can be acquired inside your AWS account. Visit <a href="https://aws.amazon.com/" target='_blank' rel='noopener'>https://aws.amazon.com/</a> and sign in to your account.

In the top right corner, click the dropdown under your username and select **My Security Credentials**.

![Screenshot of AWS site My Security Credentials page](https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/aws-secrurity-credentials-arrow.png "doc-image")

Under "My Security Credentials" you'll see section titled "Access keys for CLI, SDK, & API access". If you don't yet have an access key listed, create one.

![Screenshot of Access Keys section of AWS site My Security Credentials page](https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/aws-access-key.png "doc-image")

Save the key information that this generates, and keep it in a safe place!

### Add AWS Credentials to Saturn Cloud
Sign in to your Saturn Cloud account and select **Credentials** from the menu on the left.

<img src="https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/saturn-credentials-arrow.jpeg" style="width:200px;" alt="Saturn Cloud left menu with arrow pointing to Credentials tab" class="doc-image">

This is where you will add your S3 API key information. *This is a secure storage location, and it will not be available to the public or other users without your consent.*

At the top right corner of this page, you will find the **New** button. Click here, and you will be taken to the Credentials Creation form. 

![Screenshot of Saturn Cloud Create Credentials form](https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/credentials.jpg "doc-image")

You will be adding three credentials items: your AWS Access Key, AWS Secret Access Key, and you default region. Complete the form one time for each item. 

|  Credential | Type  | Name| Variable Name  |
|---|---|---|---|
| AWS Access Key ID   |  Environment Variable  | `aws-access-key-id` | `AWS_ACCESS_KEY_ID`
| AWS Secret Access Key | Environment Variable  | `aws-secret-access-key`  | `AWS_SECRET_ACCESS_KEY`
| AWS Default Region  | Environment Variable  | `aws-default-region`  | `AWS_DEFAULT_REGION`

Copy the values from your AWS console into the *Value* section of the credential creation form. The credential names are recommendations; feel free to change them as needed for your workflow. You must, however, use the provided *Variable Names* for S3 to connect correctly.

With this complete, your S3 credentials will be accessible by Saturn Cloud resources! You will need to restart any Jupyter Server or Dask Clusters for the credentials to populate to those resources.

<a id='connect-via-s3fs'></a>

### Connect to Data Via `s3fs`
#### Set Up the Connection
Normally, `s3fs` will automatically seek your AWS credentials from the environment. Since you have followed our instructions above for adding and saving credentials, this will work for you! 

If you don't have credentials and are accessing a public repository, set `anon=True` in the `s3fs.S3FileSystem()` call.

In [ ]:
import s3fs

s3 = s3fs.S3FileSystem(anon=False)

At this point, you can reference the `s3` handle and look at the contents of your S3 bucket as if it were a local file system. For examples, you can visit the <a href="https://s3fs.readthedocs.io/en/latest/#examples" target='_blank' rel='noopener'>s3fs documentation</a> where they show multiple ways to interact with files like this.

#### Load a CSV Using pandas
This approach just uses routine pandas syntax.

In [ ]:
import pandas as pd

file = "nyc-tlc/trip data/yellow_tripdata_2019-01.csv"
with s3.open(file, mode="rb") as f:
    df = pd.read_csv(f, parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

For small files, this approach will work fine. For large or multiple files, we recommend using Dask, as described next.

#### Load a CSV Using Dask
This syntax is the same as pandas, but produces a distributed data object.

In [ ]:
import dask.dataframe as dd

file = "nyc-tlc/trip data/yellow_tripdata_2019-01.csv"
with s3.open(file, mode="rb") as f:
    df = dd.read_csv(f, parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

#### Load a Folder of CSVs Using Dask
Dask can read and load a whole folder of files if they are formatted the same, using glob syntax.

In [ ]:
files = s3.glob("s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv")
taxi = dd.read_csv(
    files,
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": False},
    assume_missing=True,
)